### CS 210: Data Management for Data Science
#### Fall 2023

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Arnav Dashaputra"
COLLABORATORS = ""

---

## Question 1


Suppose we have a SQL database that contains a table called student_info. You should run the following code block to create the database and the table.

In [ ]:
import sqlite3

conn = sqlite3.connect('test.db')
print("Opened database successfully");

conn.execute('''
CREATE TABLE IF NOT EXISTS student_info(first_name text,
                      last_name text,
                      credit integer,
                      gpa float);''')
print("Table created successfully");
# Clearing the table
conn.execute('DELETE FROM student_info;',);

# Inserting values
conn.execute("INSERT INTO student_info VALUES('Kate', 'Perry', 120, 3.3);")
conn.execute("INSERT INTO student_info VALUES('Kelvin', 'Harris', 50, 3.0);")
conn.execute("INSERT INTO student_info VALUES('Bin', 'Diesel', 250, 3.5);")
conn.execute("INSERT INTO student_info VALUES('nick', 'Cage', 22, 2.8);")
conn.execute("INSERT INTO student_info VALUES('Shawn', 'Carter', 100, 3.7);")
conn.execute("INSERT INTO student_info VALUES('Lucy', 'Lu', 200, 3.8 );")
conn.execute("INSERT INTO student_info VALUES('John', 'Senna', 0, 0.0 );")
conn.execute("INSERT INTO student_info VALUES('Syd', 'Barrett', 183, 2.8 );")
conn.execute("INSERT INTO student_info VALUES('Peter', 'Chao', 111, 2.3 );")
conn.execute("INSERT INTO student_info VALUES('Shang', 'abi', 64, 3.1 );")

conn.commit()
conn.close()




This is an example of how to execute sql query

In [ ]:
conn = sqlite3.connect('test.db')


# cursor = conn.execute(''' Your SQL Query''')


cursor = conn.execute(''' SELECT *
                          FROM student_info;''')


for row in cursor:
  print(row)
conn.close()

Now, please write an SQL query to retrieve the first names of students whose credits are fewer than 150 and who have a GPA higher than 3.0. Then, display the results as shown in the example.

In [ ]:
def retrieve_students():
    conn = sqlite3.connect('test.db')
    
    cursor = conn.execute("""
    SELECT first_name
    FROM student_info
    WHERE credit < 150 AND gpa > 3.0;
    """)

    for row in cursor:
        print(row[0])

    conn.close()
    
retrieve_students()

## Question 2

Many people in the Computer Science department use '[dot]' and '[at]' to replace the symbols '.' and '@' when posting their emails on websites. For example, they write 'John[dot]Wick[at]rutgers[dot]edu' instead of 'John.Wick@rutgers.edu.' This practice helps mitigate issues related to email scraping and spam. Suppose we have collected all the email addresses from the Computer Science department and now want to use a regular expression to extract them.

In [ ]:
email_list = ['John[dot]Wick[at]rutgers[dot]edu',
              'Nancy@rutgers.edu.com',
              'Toby.Chavez.edu',
              'dfe.edu'
              'Steve[at]Peterson[at]rutgers[dot]edu',
              'Sydney[at]Lucas[at]rutgers[dot]edu',
              'Sydney[at][at]rutgers[dot]edu',
              'Byron.Dennis@umd.edu',
              'Nancy.Ruell@rutgers.edu',
              'Benjamin[dot]Conner[at]rutgers[dot]edu',
              'Nancy@rutgersedu',
              'dfe.edu.com',
              'dfe.edu.[]',
            ]

Write code to read each email address and extract all valid ones. Valid email addresses are those in the formats 'first.last@domain.com', 'first.last@domain.edu', 'first[dot]last[at]domain[dot]com', and 'first[dot]last[at]domain[dot]edu'. Match all valid email addresses and convert them into the format 'firstname.lastname@domain.edu' or 'firstname.lastname@domain.com'. Then, return all valid email addresses in a list.

In [ ]:
import re

def valid_email_list(email_list):
    valid_emails = []

    for email in email_list:
        if '[dot]' in email or '[at]' in email:
            email = email.replace('[dot]', '.').replace('[at]', '@') #replace [dot] and [at] with proper symbols
        
        match = re.match(r'([a-zA-Z0-9_.+-]+)\.([a-zA-Z0-9_.+-]+)@([a-zA-Z0-9_.+-]+\.(edu|com))', email)
        
        if match:
            valid_emails.append(email)

    return valid_emails

In [ ]:
assert len(valid_email_list(email_list)) == 4
assert 'John.Wick@rutgers.edu' in valid_email_list(email_list)
assert 'Nancy.Ruell@rutgers.edu' in valid_email_list(email_list)
assert 'Benjamin.Conner@rutgers.edu' in valid_email_list(email_list)
assert 'Byron.Dennis@umd.edu' in valid_email_list(email_list)

## Problem 3: EU Cities Temperatures Dataset

Given a CSV data file as represented by the sample file *EuCitiesTemperatures.csv* (213 records), load it into a Pandas DataFrame and perform the following tasks on it.

## Preprocessing/Analysis

1.Fill in the missing latitude and longitude values by calculating the average for that country. Round the average to 2 decimal places.


In [ ]:
import pandas as pd

df = pd.read_csv('./EuCitiesTemperatures.csv')

country_avg_location = df.groupby('country')[['latitude', 'longitude']].mean().round(2)

for country in country_avg_location.index:
    df.loc[(df['country'] == country) & (df['latitude'].isna()), 'latitude'] = country_avg_location.loc[country, 'latitude']
    df.loc[(df['country'] == country) & (df['longitude'].isna()), 'longitude'] = country_avg_location.loc[country, 'longitude']

print(df)

2.Find out the subset of cities that lie between latitudes 40 to 60 (both inclusive) and longitudes 15 to 30 (both inclusive). Find out which countries have the maximum number of cities in this geographical band. (More than one country could have the maximum number of values.)

In [ ]:
filtered_cities = df[(df['latitude'] >= 40) & (df['latitude'] <= 60) & (df['longitude'] >= 15) & (df['longitude'] <= 30)]
city_count_per_country = filtered_cities['country'].value_counts()
max_cities = city_count_per_country.max()
countries_with_max_cities = city_count_per_country[city_count_per_country == max_cities].index.tolist()

print(countries_with_max_cities)

3.Fill in the missing temperature values by the average temperature value of the similar region type. A region type would be a combinaton of whether it is in EU (yes/no) and whether it has a coastline (yes/no).
For example, if we have a missing temperature value for Bergen, Norway, which is not in the EU but lies on the coast, we will fill it with the average temperature of cities with EU='no' and coastline='yes'

In [ ]:
avg_temp_by_type = df.groupby(['EU', 'coastline'])['temperature'].mean()

def fill_missing_temperature(row):
    if pd.isna(row['temperature']):
        return avg_temp_by_type[row['EU'], row['coastline']]
    else:
        return row['temperature']

df['temperature'] = df.apply(fill_missing_temperature, axis=1)

print(df)

## Visualization
For all plots, make sure to label the axes, and set appropriate tick labels.

1.Plot a bar chart for the number of cities belonging to each of the regions described in Preprocessing/Analysis #3 above.

In [ ]:
import matplotlib.pyplot as plt

region_city_count = df.groupby(['EU', 'coastline'])['city'].count()
labels = [f'EU: {eu}, Coastline: {coastline}' for eu, coastline in region_city_count.index]
values = region_city_count.values

plt.figure(figsize=(10, 6))
plt.bars = plt.bar(labels, values)
plt.xlabel('Region Type')
plt.ylabel('Number of Cities')
plt.title('Number of Cities per Region Type')

plt.show()

2.Plot a scatter plot of latitude (y-axis) v/s longitude (x-axis) values to get a map-like visual of the cities under consideration. All the cities in the same country should have the same color.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

countries = df['country'].unique()
colors = plt.cm.jet(np.linspace(0, 1, len(countries))) #jet colormap array
country_to_color = dict(zip(countries, colors))

plt.figure(figsize=(12, 8))

for country in countries:
    country_data = df[df['country'] == country]
    #create a plot per country and use the corresponding color
    plt.scatter(country_data['longitude'], country_data['latitude'], color=country_to_color[country], label=country)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Scatter Plot of Cities by Country')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=2) #move legend to bottom of plot

plt.show()

3.The population column contains values unique to each country. So two cities of the same country will show the same population value. Plot a histogram of the number of countries belonging to each population group: split the population values into 5 bins (groups).

In [ ]:
import matplotlib.pyplot as plt

unique_countries = df.drop_duplicates(subset='country')

plt.figure(figsize=(10, 6))
plt.hist(unique_countries['population'], bins=5, edgecolor='black')

plt.xlabel('Population')
plt.ylabel('Number of Countries')
plt.title('Number of Countries in Each Population Group')

plt.show()

4.Plot subplots (2, 2), with proper titles, one each for the region types described in Preprocessing/Analysis #3 above.

Each subplot should be a scatter plot of Latitude (y-axis) vs. City (x-axis), where the color of the plot points should be based on the temperature values: ‘red’ for temperatures above 10, ‘blue’ for temperatures below 6 and ‘orange for temperatures between 6 and 10 (both inclusive). For each subplot, set xticks to an array of numbers from 0 to n-1 (both inclusive), where n is the total number of cities in each region type. This represents each city as a number between 0 and n-1.

In [ ]:
import matplotlib.pyplot as plt

region_types = df.groupby(['EU', 'coastline']).groups.keys() #assigns EU or Coastline vals to region_types
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten() #converts array of axes into 1D array

def color_based_on_temp(temp):
    if temp > 10:
        return 'red'
    elif temp < 6:
        return 'blue'
    else:
        return 'orange'

for i, region in enumerate(region_types): #to write each subplot individually
    region_data = df[(df['EU'] == region[0]) & (df['coastline'] == region[1])]
    cities = range(len(region_data))
    colors = region_data['temperature'].apply(color_based_on_temp)
    axes[i].scatter(cities, region_data['latitude'], color=colors)
    axes[i].set_xticks(cities)
    axes[i].set_xlabel('City (numbered)')
    axes[i].set_ylabel('Latitude')
    axes[i].set_title(f'Region: EU={region[0]}, Coastline={region[1]}')

plt.show()